<a href="https://colab.research.google.com/github/BMateusSs/Projeto-PISI3/blob/main/exploratory_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies as needed:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "winemag-data-130k-v2.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "zynicide/wine-reviews",
  file_path
)

/tmp/ipykernel_70058/4183834007.py:9: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Limpeza dos dados

In [2]:
initial_lines_quantity = len(df['description'])
print(initial_lines_quantity)
df.head()

129971


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
# Retirar linhas incompletas
df.dropna(axis="index", how= "any", subset=["price", "points", "region_1", "country", "variety", "designation"], inplace=True)
final_lines_quantity = len(df["description"])
print(final_lines_quantity)

70175


In [ ]:
# Se for feito a limpeza das descrições para tirar stop words e pontuação
# sentence transformer já lida com o stop words e pontuação
import nltk
from nltk.corpus import stopwords
import string

# Baixe as stopwords se ainda não as tiver
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Converter para lowercase
    text = text.lower()
    # Remover pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remover stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Aplicar a função de limpeza na coluna 'description'
df['description'] = df['description'].apply(clean_text)
df.head()

In [ ]:
# Usando o modelo paraphrase-multilingual-mpnet-base-v2
from sentence_transformers import SentenceTransformer
# Experimente com um modelo diferente, por exemplo:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(df['description'].tolist(), show_progress_bar=True)

In [ ]:
# Escrever os embeddings no disco
# Para não calcular eles novamente
def write_numpy_file(write_path, content):
    np.save(write_path, content)

write_numpy_file(write_path="embeddings_paraphrase_multilingual_mpnet_base_v2.npy", embeddings)

In [7]:
# Carregar os embeddings que estaram salvo em disco

import numpy as np

load_path = "embeddings_paraphrase_multilingual_mpnet_base_v2.npy"

# Carregar os embeddings
embeddings = np.load(load_path, allow_pickle=True)
print(embeddings)
print(embeddings.shape)

UnpicklingError: Failed to interpret file 'embeddings_paraphrase_multilingual_mpnet_base_v2.npy' as a pickle

In [ ]:
import umap
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
umap_2d = reducer.fit_transform(embeddings)


In [ ]:
# If you want to use t-SNE instead of UMAP, you can do it like this:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
tsne_2d = tsne.fit_transform(embeddings)


In [ ]:
# descobrir qual a quantidade clusteres que eu preciso utilizar no kmeans++
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

wcss = []
K = range(2, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(umap_2d)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(K, wcss, 'bo-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('WCSS')
plt.title('Método do Cotovelo para KMeans++')
plt.grid(True)
plt.show()

In [ ]:
# Após escolher o melhor k (exemplo: k=5)
best_k = 4
kmeans = KMeans(n_clusters=best_k, init='k-means++', random_state=42)
clusters = kmeans.fit_predict(umap_2d)
df['clusters'] = clusters

In [ ]:
import plotly.express as px
import pandas as pd

# Supondo que umap_2d, clusters e df já estão definidos
df_plot = pd.DataFrame({
    'UMAP1': umap_2d[:, 0],
    'UMAP2': umap_2d[:, 1],
    'Cluster': clusters.astype(str),  # Para colorir por cluster
    'Região': df['region_1'].values
})

fig = px.scatter(
    df_plot, x='UMAP1', y='UMAP2',
    color='Cluster',
    hover_data=['Região'],
    title='Clusters de Embeddings (UMAP + KMeans++)'
)
fig.show()

In [ ]:
# Existe diversidade na pontuação dos clusters
media_pontos_por_cluster = df.groupby('cluster')['points'].mean()

print(media_pontos_por_cluster)
# A média da qualidade so vinho não tem relação com o cluster

In [ ]:
# Existe diversidade de preço entre os clusteres
media_preco_por_cluster = df.groupby('cluster')['price'].mean()

print(media_preco_por_cluster)

In [ ]:
def cross_columns_with_cluster(column: str):
        # Tem diversidade de países em cada cluster
    clusters_por_pais = df.groupby(column)['cluster'].nunique().sort_values(ascending=False)
    print(clusters_por_pais)

    # Para visualizar a distribuição detalhada:
    distribuicao = pd.crosstab(df[column], df['cluster'])
    print(distribuicao)

In [ ]:
cross_columns_with_cluster('country')

In [ ]:
print(len(df['designation']))

In [ ]:
# Não existe diversidade nos clusteres de um mesmo tipo de vinho
import matplotlib.pyplot as plt
import pandas as pd

# Passo 1: Conta em quantos clusters cada designation aparece
clusters_por_designation = df.groupby('designation')['cluster'].nunique()

# Passo 2: Conta quantas designations aparecem em 1, 2, 3 ou 4 clusters
contagem = clusters_por_designation.value_counts().sort_index()
# Garante que todos os valores de 1 a 4 estejam presentes
contagem = contagem.reindex([1, 2, 3, 4], fill_value=0)

import matplotlib.pyplot as plt

# clusters_por_designation e contagem já definidos como no seu código

plt.figure(figsize=(7, 4))
bars = plt.bar(contagem.index, contagem.values, color='#4F81BD')
plt.xlabel('Número de clusters em que a designation aparece')
plt.ylabel('Quantidade de designations')
plt.title('Distribuição de Designations por Número de Clusters')
plt.xticks(contagem.index)
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Adiciona os valores acima das barras
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 10, int(yval), ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def quantity_per_cluster(col: str):
    # Seleciona as 10 designations mais populares
    top_designations = df[col].value_counts().nlargest(10).index
    # Filtra o DataFrame para essas designations
    df_top = df[df[col].isin(top_designations)]
    # Cria a tabela dinâmica: linhas = designation, colunas = cluster, valores = contagem de vinhos
    pivot = pd.pivot_table(
        df_top,
        index=col,
        columns='cluster',
        values='country',  # pode ser qualquer coluna, pois o que importa é o count
        aggfunc='count',
        fill_value=0
    )

    # Ordena as designations pela popularidade
    pivot = pivot.loc[top_designations]

    # Exibe a tabela com Matplotlib
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.axis('off')
    tabela_plot = ax.table(
        cellText=pivot.values,
        rowLabels=pivot.index,
        colLabels=[f'Cluster {c}' for c in pivot.columns],
        cellLoc='center',
        loc='center'
    )
    tabela_plot.auto_set_font_size(False)
    tabela_plot.set_fontsize(12)
    tabela_plot.scale(1.2, 1.2)
    plt.title(f'Top 10 {col}: Nº de Vinhos por Cluster', fontsize=14, pad=20)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

quantity_per_cluster('designation')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

quantity_per_cluster('variety')

In [ ]:
quantity_per_cluster('taster_twitter_handle')

In [ ]:
import matplotlib.pyplot as plt

# Contar as 10 regiões mais frequentes
top_regions = df['region_1'].value_counts().nlargest(10).index

# Filtrar o DataFrame e os embeddings para essas regiões
mask = df['region_1'].isin(top_regions)
filtered_x = x_coords[mask]
filtered_y = y_coords[mask]
filtered_regions = df['region_1'][mask].reset_index(drop=True)

# Plotar apenas as 10 regiões mais frequentes
unique_regions = filtered_regions.unique()
colors = plt.cm.get_cmap('tab10', len(unique_regions))

plt.figure(figsize=(10, 8))
for i, region in enumerate(unique_regions):
    region_mask = filtered_regions == region
    plt.scatter(
        filtered_x[region_mask],
        filtered_y[region_mask],
        label=region,
        color=colors(i),
        s=10,
        alpha=0.6
    )

plt.title('Visualização UMAP das 10 Regiões com Mais Dados')
plt.xlabel('UMAP Componente 1')
plt.ylabel('UMAP Componente 2')
plt.legend(title='Região', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()
# Existe vinhos de uma mesma região em vários clusteres diferentes

In [ ]:
from collections import Counter

def get_top_n_words(corpus, n=None):
    words = [word for sentence in corpus for word in sentence.split()]
    word_counts = Counter(words)
    return word_counts.most_common(n)

# Exemplo: Top 20 palavras para cada cluster
for cluster_id in sorted(df['cluster'].unique()):
    print(f"\nTop palavras para o Cluster {cluster_id}:")
    cluster_descriptions = df[df['cluster'] == cluster_id]['description']
    top_words = get_top_n_words(cluster_descriptions, n=20)
    print(top_words)

In [ ]:
print(len(embeddings))